In [39]:
from bs4 import BeautifulSoup
import bs4
print("bs4 v.{}".format(bs4.__version__))
import requests
print('requests v.{}'.format(requests.__version__))
import pandas as pd
print('pandas v.{}'.format(pd.__version__))

bs4 v.4.5.3
requests v.2.12.4
pandas v.0.19.2


In [75]:
url = 'http://beascout.scouting.org'

In [76]:
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

In [77]:
r = requests.get(url, headers = hdr)

In [78]:
soup = BeautifulSoup(r.content,'html.parser')

In [79]:
soup


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head id="Head1">
<script src="//cdn.optimizely.com/js/4889642714.js"></script>
<!-- Hotjar Tracking Code for http://beascout.scouting.org -->
<script>
    (function(h,o,t,j,a,r){
        h.hj=h.hj||function(){(h.hj.q=h.hj.q||[]).push(arguments)};
        h._hjSettings={hjid:149458,hjsv:5};
        a=o.getElementsByTagName('head')[0];
        r=o.createElement('script');r.async=1;
        r.src=t+h._hjSettings.hjid+j+h._hjSettings.hjsv;
        a.appendChild(r);
    })(window,document,'//static.hotjar.com/c/hotjar-','.js?sv=');
</script>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/><title>
	Join Cub Scouts And Boy Scouts Of America
</title><meta content="Be A S

In [80]:
links = soup.find_all("a", href=True)
found_links=[]
for link in links: 
    if link['href'][:4] == 'http':
        if link['href'] not in found_links:
            print(link['href'])
            found_links.append(link['href'])
            link_count+=1
print(link_count)

http://www.scouting.org
http://scouting.org
http://aplacetogive.scouting.org/
https://beascout.scouting.org/?sc_lang=es-MX
http://www.scouting.org/meritbadges.aspx
http://beascout.scouting.org/BeAScoutMap.aspx
http://blog.scoutingmagazine.org/2013/05/21/ok/
https://instagram.com/boyscoutsofamerica/
http://www.flickr.com/photos/boyscoutsofamerica
http://www.youtube.com/bsa100years
http://twitter.com/boyscouts
http://www.facebook.com/pages/Boy-Scouts-of-America/113441755297
http://www.scouting.org/PrivacyPolicy.aspx
http://www.scouting.org/TermsAndConditions.aspx
14


In [82]:
print("cookie count: {}".format(len(r.cookies)))
for cookie in r.cookies:
    print(cookie)

cookie count: 2
<Cookie .ASPXANONYMOUS=R0f-zd7k0gEkAAAAN2NkMGZhMzQtOTNiZS00ODYyLTlkZWUtODliOTAwYjdkOTc4zOmLGc4x7J7WeeLgvcA-LFRY55g1 for beascout.scouting.org/>
<Cookie ASP.NET_SessionId=dr3asmcehxrchgqghxf4xb1h for beascout.scouting.org/>


In [85]:
r.encoding

'utf-8'

In [84]:
r.status_code

200

In [83]:
r.history

[<Response [302]>]

# Crawling Website's HREF links

In [72]:
working_urls = ["http://www.scouting.org"]
completed_urls = []
iterations = 0
limit = 100
records = [['url','cookies','encoding','status_code','title','links']]

hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

while working_urls and iterations <= limit:
    iterations +=1
    
    current_url = working_urls.pop(0)
    current_record = []
    
    r = requests.get(current_url, headers = hdr)
    soup = BeautifulSoup(r.content,'html.parser')
    
    current_record.append(current_url)
    current_record.append(len(r.cookies))
    current_record.append(r.encoding)
    current_record.append(r.status_code)
    
    titles = soup.find_all("title")
    title_text = ''
    for title in titles:
        title_text+=title.get_text()
    
    current_record.append(title_text)
    
    links = soup.find_all("a", href=True)
    link_count = 0
    for link in links: 
        if link['href'][:4] == 'http' and link['href'] not in working_urls and link['href'] not in completed_urls and link['href']!=current_url:
            working_urls.append(link['href'])
        link_count+=1
    current_record.append(link_count)
    
    completed_urls.append(current_url)
    
    records.append(current_record)

In [74]:
df = pd.DataFrame(records[1:], columns = records[0])
print(len(df))
df.head()

101


,url,cookies,encoding,status_code,title,links
0,http://www.scouting.org,0,utf-8,200,\r\n\tBoy Scouts of America | Prepared. For Li...,75
1,https://beascout.scouting.org,2,utf-8,200,\r\n\tJoin Cub Scouts And Boy Scouts Of Americ...,45
2,http://www.scouting.org/Youth.aspx,0,utf-8,200,\r\n\tBegin the Adventure\r\n,71
3,http://www.scouting.org/Parent.aspx,0,utf-8,200,\r\n\tProviding a Compass for Life\r\n,79
4,http://www.scouting.org/Volunteer.aspx,0,utf-8,200,\r\n\tShaping America's Future\r\n,73
